In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# change the directory in Google Drive for the training data

hawker_dir = '/content/drive/My Drive/hawker'

from google.colab import drive
drive.mount('/content/drive')

class_labels = os.listdir(hawker_dir)

class_labels.sort()


print(class_labels)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['bak_chor_mee', 'chicken_rice', 'curry_puff', 'fish_soup', 'kaya_toast', 'laksa']


In [ ]:
data_generator = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values between 0 and 1
   # featurewise_center=True,
    rotation_range=300,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 10%
    height_shift_range=0.2,  # Randomly shift images vertically by 10%
    shear_range=0.2,  # Apply shear transformation with a shear intensity of 0.2
    zoom_range=0.2,  # Randomly zoom images by 20%
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True,
    validation_split=0.3, #split is 0.3 of the data for verification
    fill_mode='nearest'  # Fill any newly created pixels after rotation or shifting
)

#print(f"train_hawker_dir:" +train_hawker_dir)
#print(f"valid_hawker_dir:" +valid_hawker_dir)

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator

print(hawker_dir)
train_generator = data_generator.flow_from_directory(
       # train_hawker_dir,  # This is the source directory for training images
        hawker_dir,
        target_size=(224, 224),# All images will be resized to 300x300
        batch_size=16,
        class_mode='categorical'
        # Since we use binary_crossentropy loss, we need binary labels
       )

validation_generator = validation_datagen.flow_from_directory(
        #validate_hawker_dir,  # This is the source directory for validation images

        hawker_dir,
        target_size=(224, 224),# All images will be resized to 300x300
        batch_size=16,
        class_mode='categorical'
        # Since we use binary_crossentropy loss, we need binary labels
        )


#https://drive.google.com/drive/folders/1a1p31ogK-IspJCOiliX9X9zx9ESwBM5v?usp=sharing


/content/drive/My Drive/hawker
Found 4788 images belonging to 6 classes.
Found 4788 images belonging to 6 classes.


In [ ]:


pre_trained_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')



x= layers.GlobalAveragePooling2D()(pre_trained_model.output)

x= layers.Dense(128,activation='relu')(x)

# change the drop out
x = layers.Dropout(0.2)(x)

# change the regularization weight
x=layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)

x = layers.Dense  (len(class_labels), activation='softmax')(x)
#x=layers.Dense(len(class_labels))(x)

model = Model( pre_trained_model.input, x)

# Set training parameters
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
# Constant for epochs
EPOCHS = 75

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=100, #400 images = batch_size * step 400= 10
      epochs=EPOCHS,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=2
)

# save the model as h5 format

model.save('/content/drive/My Drive/models/makan_mobilenet.h5')


# plot out the

import matplotlib.pyplot as plt

# Plot the model results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()



Epoch 1/25
200/200 [==============================] - 1046s 5s/step - loss: 1.2906 - accuracy: 0.7138 - val_loss: 1.2418 - val_accuracy: 0.8125
Epoch 2/25
200/200 [==============================] - 668s 3s/step - loss: 0.7351 - accuracy: 0.9115 - val_loss: 1.3511 - val_accuracy: 0.7812
Epoch 3/25
200/200 [==============================] - 659s 3s/step - loss: 0.6309 - accuracy: 0.9328 - val_loss: 0.9072 - val_accuracy: 0.8438
Epoch 4/25
200/200 [==============================] - 655s 3s/step - loss: 0.5433 - accuracy: 0.9469 - val_loss: 0.7760 - val_accuracy: 0.8750
Epoch 5/25
200/200 [==============================] - 649s 3s/step - loss: 0.5052 - accuracy: 0.9583 - val_loss: 0.5223 - val_accuracy: 0.9375
Epoch 6/25
200/200 [==============================] - 653s 3s/step - loss: 0.4758 - accuracy: 0.9581 - val_loss: 0.3634 - val_accuracy: 1.0000
Epoch 7/25
200/200 [==============================] - 645s 3s/step - loss: 0.4311 - accuracy: 0.9642 - val_loss: 0.6029 - val_accuracy: 0.875